In [ ]:
# libraries
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datasage.styles import DARK_THEME,LIGHT_THEME,theme
from datasage.core import Leonardo
import matplotlib.colors
import matplotlib.font_manager as fm
import mplcyberpunk
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
leo = Leonardo()
plt.style.use(LIGHT_THEME)

In [ ]:
# settings
font_family = leo.setup_google_font()
plt.rcParams['font.family'] = font_family
# plt.rcParams['figure.dpi'] = 300

# palette
colors = ['#949799', '#242728', '#0085a1']
ccmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
sns.palplot(colors,size=1)
plt.title('Palette and font',size=9)
plt.show()

## Ipotesi Nulla e Alternativa

$H_0: \mu_1 = \mu_2 = ... = \mu_k$ (tutte le medie dei gruppi sono uguali)

$H_1:$ almeno una media è diversa dalle altre

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
# Calcolo statistiche descrittive
region_stats = df.groupby('region')['charges'].describe()
print("Statistiche descrittive per regione:")
print(region_stats)

# Visualizzazione con boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='region', y='charges', data=df)
plt.title('Distribuzione dei costi per regione')
plt.ylabel('Charges ($)')
plt.show()

# Esecuzione one-way ANOVA
regions = [group for name, group in df.groupby('region')['charges']]
f_statistic, p_value = stats.f_oneway(*regions)

print("\nRisultati One-way ANOVA per regione:")
print(f"F-statistic: {f_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Test di Tukey per confronti multipli
tukey = pairwise_tukeyhsd(df['charges'], df['region'])
print("\nRisultati test di Tukey:")
print(tukey)

# 2. Two-way ANOVA: Charges by Region and Smoker
#---------------------------------------------

# Creazione del modello
model = ols('charges ~ C(region) + C(smoker) + C(region):C(smoker)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nRisultati Two-way ANOVA:")
print(anova_table)

# Visualizzazione dell'interazione
plt.figure(figsize=(10, 6))
sns.boxplot(x='region', y='charges', hue='smoker', data=df)
plt.title('Distribuzione dei costi per regione e stato fumatore')
plt.ylabel('Charges ($)')
plt.show()

# Plot dell'interazione
plt.figure(figsize=(10, 6))
interaction_data = df.groupby(['region', 'smoker'])['charges'].mean().unstack()
interaction_data.plot(marker='o')
plt.title('Plot di interazione: Region x Smoker')
plt.ylabel('Media delle charges ($)')
plt.grid(True)
plt.show()

# 3. ANCOVA: Charges by Smoker Status with Age as covariate
#-------------------------------------------------------

# Creazione del modello ANCOVA
ancova_model = ols('charges ~ C(smoker) + age', data=df).fit()
ancova_table = sm.stats.anova_lm(ancova_model, typ=2)
print("\nRisultati ANCOVA:")
print(ancova_table)

# Visualizzazione della relazione tra age e charges per fumatori e non fumatori
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='age', y='charges', hue='smoker', alpha=0.5)
sns.regplot(data=df[df['smoker']=='yes'], x='age', y='charges', scatter=False, color='blue')
sns.regplot(data=df[df['smoker']=='no'], x='age', y='charges', scatter=False, color='orange')
plt.title('Relazione tra età e costi per stato fumatore')
plt.ylabel('Charges ($)')
plt.show()

# 4. Assunzioni ANOVA
#-------------------

# Test di normalità per ogni gruppo
for region in df['region'].unique():
    stat, p_value = stats.shapiro(df[df['region'] == region]['charges'])
    print(f"\nTest di Shapiro-Wilk per {region}:")
    print(f"Statistica: {stat:.4f}")
    print(f"p-value: {p_value:.4f}")

# Test di Levene per l'omogeneità della varianza
stat, p_value = stats.levene(*regions)
print("\nTest di Levene per l'omogeneità della varianza:")
print(f"Statistica: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Q-Q plot per ogni regione
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.ravel()

for idx, region in enumerate(df['region'].unique()):
    stats.probplot(df[df['region'] == region]['charges'], dist="norm", plot=axes[idx],color='#0085a1')
    axes[idx].set_title(f'Q-Q Plot per {region}')

plt.tight_layout()
plt.show()
